# ランダムカットフォレストモデルの学習
## ライブラリの読み込み

In [ ]:
import boto3
import sagemaker
from sagemaker.estimator import Estimator
from sagemaker.amazon.amazon_estimator import get_image_uri

##  パラメータ

In [ ]:

train_s3_path = 's3://bucket-name/sagemaker/iot-analytics/machine-temperature/train.csv'
test_s3_path = 's3://bucket-name/sagemaker/iot-analytics/machine-temperature/test.csv'

# 学習とエンドポイントの展開を行う際に使うIAMロール名
execution_role = sagemaker.get_execution_role()

hyperparameters = dict(
    num_samples_per_tree=256,
    num_trees=100,
    feature_dim=12*24
)

model_artifact_path = 's3://bucket-name/sagemaker/iot-analytics/machine-temperature/'
base_job_name = 'rcf_iot_analytics'

## 学習

In [ ]:

# ランダムカットフォレスト用のコンテナイメージ
training_image = get_image_uri(boto3.Session().region_name, 'randomcutforest')

# 学習用処理の設定
rcf = Estimator(
    role=execution_role,
    train_instance_count=1,
    train_instance_type='ml.m4.xlarge',
    output_path=model_artifact_path,
    base_job_name=base_job_name,
    image_name=training_image
)

# ハイパーパラメータの設定
rcf.set_hyperparameters(**hyperparameters)

# 教師データ
train_s3_data = sagemaker.s3_input(
   s3_data = train_s3_path,
   content_type = 'text/csv;label_size=0',
   distribution = 'ShardedByS3Key'
)

# テストデータ
test_s3_data = sagemaker.s3_input(
   s3_data = test_s3_path,
   content_type = 'text/csv;label_size=1',
   distribution = 'FullyReplicated'
)


# 学習開始
rcf.fit({'train': train_s3_data, 'test': test_s3_data}, wait=True)


## ジョブ名を保存

In [ ]:
import papermill as pm
pm.record('job_name', rcf.latest_training_job.name)
